In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/description.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data_solution.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt


In [2]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


In [3]:
# Download stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# Load datasets
train_data = pd.read_csv('/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt',sep=' ::: ' ,header=None ,names=['id','title','genre','plot'],engine='python')
train_data

,id,title,genre,plot
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his doc...
1,2,Cupid (1997),thriller,A brother and sister with a past incestuous re...
2,3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fiel...
3,4,The Secret Sin (1915),drama,To help their unemployed father make ends meet...
4,5,The Unrecovered (2007),drama,The film's title refers not only to the un-rec...
...,...,...,...,...
54209,54210,"""Bonino"" (1953)",comedy,This short-lived NBC live sitcom centered on B...
54210,54211,Dead Girls Don't Cry (????),horror,The NEXT Generation of EXPLOITATION. The siste...
54211,54212,Ronald Goedemondt: Ze bestaan echt (2008),documentary,"Ze bestaan echt, is a stand-up comedy about gr..."
54212,54213,Make Your Own Bed (1944),comedy,Walter and Vivian live in the country and have...


In [5]:
test_data = pd.read_csv('/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data.txt', sep=' ::: ', header=None, names=['id', 'title', 'plot'], engine='python')
test_data

,id,title,plot
0,1,Edgar's Lunch (1998),"L.R. Brane loves his life - his car, his apart..."
1,2,La guerra de papá (1977),"Spain, March 1964: Quico is a very naughty chi..."
2,3,Off the Beaten Track (2010),One year in the life of Albin and his family o...
3,4,Meu Amigo Hindu (2015),"His father has died, he hasn't spoken with his..."
4,5,Er nu zhai (1955),Before he was known internationally as a marti...
...,...,...,...
54195,54196,"""Tales of Light & Dark"" (2013)","Covering multiple genres, Tales of Light & Dar..."
54196,54197,Der letzte Mohikaner (1965),As Alice and Cora Munro attempt to find their ...
54197,54198,Oliver Twink (2007),"A movie 169 years in the making. Oliver Twist,..."
54198,54199,Slipstream (1973),"Popular, but mysterious rock D.J Mike Mallard ..."


In [6]:
test_data_solution = pd.read_csv('/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data_solution.txt', sep=' ::: ', header=None, names=['id', 'title', 'genre', 'plot'],  engine='python')
test_data_solution

,id,title,genre,plot
0,1,Edgar's Lunch (1998),thriller,"L.R. Brane loves his life - his car, his apart..."
1,2,La guerra de papá (1977),comedy,"Spain, March 1964: Quico is a very naughty chi..."
2,3,Off the Beaten Track (2010),documentary,One year in the life of Albin and his family o...
3,4,Meu Amigo Hindu (2015),drama,"His father has died, he hasn't spoken with his..."
4,5,Er nu zhai (1955),drama,Before he was known internationally as a marti...
...,...,...,...,...
54195,54196,"""Tales of Light & Dark"" (2013)",horror,"Covering multiple genres, Tales of Light & Dar..."
54196,54197,Der letzte Mohikaner (1965),western,As Alice and Cora Munro attempt to find their ...
54197,54198,Oliver Twink (2007),adult,"A movie 169 years in the making. Oliver Twist,..."
54198,54199,Slipstream (1973),drama,"Popular, but mysterious rock D.J Mike Mallard ..."


In [7]:
def preprocess(text):
    # Remove special characters and punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

In [8]:
# Apply preprocessing to the plot column
train_data['plot'] = train_data['plot'].apply(preprocess)
test_data['plot'] = test_data['plot'].apply(preprocess)

trainign a logistic regression model

In [9]:
# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

In [10]:
# Fit and transform the training data
X_train = tfidf_vectorizer.fit_transform(train_data['plot'])

In [11]:
# Transform the test data
X_test = tfidf_vectorizer.transform(test_data['plot'])


In [12]:
# Initialize label encoder
label_encoder = LabelEncoder()

In [13]:
# Fit and transform the genre column
y_train = label_encoder.fit_transform(train_data['genre'])

In [14]:
# Initialize and train the logistic regression model
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [15]:
# Predict genres on the test data
y_pred = model.predict(X_test)

In [16]:
# Convert predictions back to original genre labels
y_pred_labels = label_encoder.inverse_transform(y_pred)


In [17]:
# Load the test solution data
y_test = label_encoder.transform(test_data_solution['genre'])


In [18]:
# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.5894280442804428


In [19]:
# Print the classification report for more detailed performance metrics
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

              precision    recall  f1-score   support

      action       0.53      0.25      0.34      1314
       adult       0.63      0.19      0.30       590
   adventure       0.71      0.14      0.24       775
   animation       0.59      0.03      0.05       498
   biography       0.00      0.00      0.00       264
      comedy       0.54      0.59      0.57      7446
       crime       0.45      0.02      0.03       505
 documentary       0.66      0.88      0.75     13096
       drama       0.54      0.80      0.64     13612
      family       0.56      0.07      0.12       783
     fantasy       0.71      0.02      0.03       322
   game-show       0.92      0.47      0.62       193
     history       0.00      0.00      0.00       243
      horror       0.67      0.57      0.61      2204
       music       0.70      0.38      0.49       731
     musical       1.00      0.00      0.01       276
     mystery       0.00      0.00      0.00       318
        news       0.75    

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TARINING A NAIVE BAYES MODEL 

In [20]:
# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(train_data['plot'])
X_test_tfidf = tfidf_vectorizer.transform(test_data['plot'])

In [21]:
# Label Encoding
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(train_data['genre'])
y_test_encoded = label_encoder.transform(test_data_solution['genre'])


In [22]:
# Train the Naive Bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, y_train_encoded)

MultinomialNB()

In [23]:
# Evaluate the model on the test set
y_test_pred = nb_model.predict(X_test_tfidf)
y_test_pred_labels = label_encoder.inverse_transform(y_test_pred)


In [24]:
# Calculate and print test accuracy
print(f"Test Accuracy: {accuracy_score(y_test_encoded, y_test_pred)}")
print(classification_report(y_test_encoded, y_test_pred, target_names=label_encoder.classes_))

Test Accuracy: 0.4441328413284133
              precision    recall  f1-score   support

      action       0.00      0.00      0.00      1314
       adult       0.00      0.00      0.00       590
   adventure       0.00      0.00      0.00       775
   animation       0.00      0.00      0.00       498
   biography       0.00      0.00      0.00       264
      comedy       0.74      0.04      0.07      7446
       crime       0.00      0.00      0.00       505
 documentary       0.53      0.90      0.67     13096
       drama       0.38      0.88      0.53     13612
      family       0.00      0.00      0.00       783
     fantasy       0.00      0.00      0.00       322
   game-show       0.00      0.00      0.00       193
     history       0.00      0.00      0.00       243
      horror       0.00      0.00      0.00      2204
       music       0.00      0.00      0.00       731
     musical       0.00      0.00      0.00       276
     mystery       0.00      0.00      0.00    

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


trying a random forest classifier 

In [25]:
# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(train_data['plot'])
X_test_tfidf = tfidf_vectorizer.transform(test_data['plot'])

In [26]:
# Label Encoding
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(train_data['genre'])
y_test_encoded = label_encoder.transform(test_data_solution['genre'])

In [27]:
# Train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_tfidf, y_train_encoded)


RandomForestClassifier(random_state=42)

In [28]:
# Evaluate the model on the test set
y_test_pred = rf_model.predict(X_test_tfidf)
y_test_pred_labels = label_encoder.inverse_transform(y_test_pred)


In [29]:
# Calculate and print test accuracy
print(f"Test Accuracy: {accuracy_score(y_test_encoded, y_test_pred)}")
print(classification_report(y_test_encoded, y_test_pred, target_names=label_encoder.classes_))


Test Accuracy: 0.4875276752767528
              precision    recall  f1-score   support

      action       0.69      0.01      0.01      1314
       adult       0.63      0.08      0.14       590
   adventure       0.84      0.10      0.17       775
   animation       0.09      0.00      0.00       498
   biography       0.00      0.00      0.00       264
      comedy       0.55      0.26      0.35      7446
       crime       1.00      0.00      0.01       505
 documentary       0.55      0.89      0.68     13096
       drama       0.41      0.84      0.55     13612
      family       0.90      0.02      0.04       783
     fantasy       0.00      0.00      0.00       322
   game-show       0.88      0.47      0.61       193
     history       0.00      0.00      0.00       243
      horror       0.75      0.11      0.20      2204
       music       0.90      0.08      0.15       731
     musical       0.75      0.02      0.04       276
     mystery       1.00      0.00      0.01    

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
